<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/15%20sept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
from extract import execute
from replace_DFI_3 import findIndex3


# Rest of your import statements...

def display_dataframe_in_treeview(treeview, df):
    treeview.delete(*treeview.get_children())  # Clear existing data

    # Insert columns
    treeview["columns"] = list(df.columns)
    style = ttk.Style()
    style.configure("Treeview.Heading", background="lightblue", font=('Arial', 12, 'bold'))

    for col in df.columns:
        treeview.heading(col, text=col, anchor="center")
        treeview.column(col, anchor="center", width=150)  # Increase width for all columns

    # Insert rows
    for _, row in df.iterrows():
        treeview.insert("", "end", values=list(row))


def reset_dataframe(treeview, original_df, original_column_widths):
    display_dataframe_in_treeview(treeview, original_df)

    for col, width in original_column_widths.items():
        treeview.column(col, width=width)


def main():
    file_path = filedialog.askopenfilename()
    dataframes, res, file_path = execute(file_path)  # You need to define the execute function
    root = tk.Tk()
    root.title("Editable Dataframes")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    canvas = tk.Canvas(root)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    scrollbar_y = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
    canvas.configure(yscrollcommand=scrollbar_y.set)

    frame = ttk.Frame(canvas)
    canvas.create_window((0, 0), window=frame, anchor="nw")

    def update_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    frame.bind("<Configure>", update_scroll_region)

    treeviews = {}  # Dictionary to store treeviews for each dataframe

    for name, df in dataframes.items():
        dataframe_frame = ttk.Frame(frame, padding=5, relief=tk.RAISED, borderwidth=2)
        dataframe_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        dataframe_label = ttk.Label(dataframe_frame, text=name, font=('Arial', 14, 'bold'))
        dataframe_label.pack(pady=5)

        treeview = ttk.Treeview(dataframe_frame, show="headings", style="Treeview.Heading")
        treeview.pack(fill="both", expand=True)

        # Store original column widths
        original_column_widths = {col: 150 for col in df.columns}

        display_dataframe_in_treeview(treeview, df)

        for col in df.columns:
            treeview.column(col, anchor="center", width=original_column_widths[col])  # Set original width

        reset_button = tk.Button(dataframe_frame, text="Undo",
                                 command=lambda tv=treeview, df=df, widths=original_column_widths: reset_dataframe(tv,
                                                                                                                   df,
                                                                                                                   widths))
        reset_button.pack()

        # Create vertical scrollbar for the treeview
        scrollbar_y_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.VERTICAL, command=treeview.yview)
        scrollbar_y_treeview.pack(side=tk.RIGHT, fill=tk.Y)
        treeview.configure(yscrollcommand=scrollbar_y_treeview.set)

        # Bind mouse wheel event for vertical scrolling
        def treeview_scroll_y(event, treeview=treeview):
            treeview.yview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<MouseWheel>", treeview_scroll_y)

        # Create horizontal scrollbar for the treeview
        scrollbar_x_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.HORIZONTAL, command=treeview.xview)
        scrollbar_x_treeview.pack(side=tk.BOTTOM, fill=tk.X)
        treeview.configure(xscrollcommand=scrollbar_x_treeview.set)

        # Bind shift + mouse wheel event for horizontal scrolling
        def treeview_scroll_x(event, treeview=treeview):
            treeview.xview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<Shift-MouseWheel>", treeview_scroll_x)

        treeviews[dataframe_frame] = treeview  # Store the treeview in the dictionary

    update_scroll_region(None)
    root.lift()

    root.geometry(f"{screen_width}x{screen_height}")
    root.wm_state("zoomed")

    def canvas_scroll(event):
        if event.delta < 0:
            canvas.yview_scroll(1, "units")
        else:
            canvas.yview_scroll(-1, "units")

    def canvas_scroll_horizontal(event):
        if event.delta < 0:
            canvas.xview_scroll(1, "units")
        else:
            canvas.xview_scroll(-1, "units")

    root.bind_all("<MouseWheel>", canvas_scroll)
    root.bind_all("<Shift-MouseWheel>", canvas_scroll_horizontal)

    # Create a horizontal scrollbar at the bottom
    scrollbar_x = ttk.Scrollbar(root, orient=tk.HORIZONTAL, command=canvas.xview)
    scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)
    canvas.configure(xscrollcommand=scrollbar_x.set)

    root.mainloop()


if __name__ == '__main__':
    main()
